# **Preparing Data For Model Training**


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Helper functions
def show_min_max(array, i):
  random_image = array[i]
  print("min and max value in image: ", random_image.min(), random_image.max())


def plot_image(array, i, labels):
  plt.imshow(np.squeeze(array[i]))
  plt.title(str(label_names[labels[i]]))
  plt.xticks([])
  plt.yticks([])
  plt.show()

# Variables to keep track of image size
img_rows, img_cols = 32, 32

# Variable for amount of output classes
num_classes = 10

# Set up array for labels:
label_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

# Loading and backing up data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
(train_images_backup, train_labels_backup), (test_images_backup, test_labels_backup) = cifar10.load_data()

# Code to adjust the label arrays.
train_labels_backup = [item for sublist in train_labels_backup for item in sublist] 
test_labels_backup = [item for sublist in test_labels_backup for item in sublist] 

# Printing out the data shape
print("Training data shape:",train_images.shape) 
print("Test data shape:",test_images.shape) 

# Variable to keep track of input shape. 
input_shape = (img_rows, img_cols, 3)

# Print out the 100th image
plot_image(train_images, 100, train_labels_backup)

# Show it's min and max value
show_min_max(train_images, 100)

# Reformating the data to float32
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

# Normalize the values by dividing by 255
train_images /= 255
test_images /= 255

# Convert labels using one-hot encoding. 
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

# **Training The Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.client import device_lib

#change for your number of NVIDA GPUs (if you have more than one)
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 12} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Change this to change accuracy
# The more Epochs the higher the accuracy and longer training times
# Less Epochs means faster trainng times but lower accuracy
epochs = 20

batch_size = 128
model = Sequential()

#Layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.2))
model.add(BatchNormalization())

#Layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(rate=0.3))
model.add(BatchNormalization())

#Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(rate=0.3))
model.add(BatchNormalization())

#Layer 4
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) 
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(rate=0.3))
model.add(BatchNormalization())

#Output Layer
model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

model.summary()

#Compiling and Training Network
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_images, test_labels), shuffle=True)
scores = model.evaluate(test_images, test_labels,verbose=0)
print('Test accuracy:', scores[1])
model.save("Cifar10_Model.h5")

# **Testing The Model**

In [ ]:
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import random

num = 0
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
(train_images_backup, train_labels_backup), (test_images_backup, test_labels_backup) = cifar10.load_data()

def GetRandomImage():
    global num
    num = random.randint(0, 10000)
    plt.imshow(test_images[num])
    plt.xticks([])
    plt.yticks([])
    plt.savefig("./img.png", bbox_inches='tight',pad_inches = 0)
    plt.show()
    
GetRandomImage()
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('Cifar10_Model.h5')
labels = [line for line in open("CifarLabels.txt")]

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 32, 32, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('img.png')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (32, 32)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 255) - 1

 # Load the image into the array
data[0] = test_images[num]

# run the inference
prediction = model.predict(data)
print("\nComputer thinks this is a:", labels[np.argmax(prediction)])
actualindex = str(test_labels[num])
actualindex = actualindex.replace("[", "", 1)
actualindex = actualindex.replace("]", "", 1)
print("This is actually a:", labels[int(actualindex)])